In [ ]:
import json
import matplotlib.pyplot as plt
log_name = "total_results.json"

with open(log_name, "r") as f:
    data = json.load(f)

print("Total results: ", len(data))

example_key = ""
for key in data.keys():
    # print(key)
    if "syntax" in data[key].keys():
        example_key = key

for key in data[example_key].keys():
    print(key)

for key in data[example_key]["syntax"].keys():
    print(key)

for key in data[example_key]["board"].keys():
    print(key)

print(data[example_key]["syntax"]["find_num_indices"])
print(data[example_key]["board"]["board_to_piece_state"])
print(data[example_key]["eval_results"])

In [ ]:
syntax_keys = {}
for func in data[example_key]["syntax"].keys():
    for var in data[example_key]["syntax"][func].keys():
        syntax_keys[var] = 0.0
    break

board_keys = {}

for func in data[example_key]["board"].keys():
    for var in data[example_key]["board"][func].keys():

        if type(data[example_key]["board"][func][var]) == list:
            continue
        if type(data[example_key]["board"][func][var]) == dict:
            continue

        board_keys[var] = 0.0
    break

print("Syntax keys: ", syntax_keys)
print("Board keys: ", board_keys)

for key in data.keys():
    if "syntax" not in data[key].keys():
        continue
    
    syntax_dict = syntax_keys.copy()
    for func in data[key]["syntax"].keys():
        for var in data[key]["syntax"][func].keys():
            if var not in syntax_dict.keys():
                continue
            syntax_dict[var] += data[key]["syntax"][func][var]
    for var in syntax_dict.keys():
        syntax_dict[var] /= len(data[key]["syntax"].keys())

    board_dict = board_keys.copy()
    for func in data[key]["board"].keys():
        for var in data[key]["board"][func].keys():
            if var not in board_dict.keys():
                continue
            board_dict[var] += data[key]["board"][func][var]
    for var in board_dict.keys():
        board_dict[var] /= len(data[key]["board"].keys())

    data[key]["syntax"]["syntax_average"] = syntax_dict
    data[key]["board"]["board_average"] = board_dict

        

In [ ]:
vars = {}

for key in data.keys():
    if "syntax" not in data[key].keys():
        continue

    vars[key] = {}

    for func in data[key]["syntax"].keys():
        for var in data[key]["syntax"][func].keys():
            vars[key][f"{func + '_' + var}"] = data[key]["syntax"][func][var]
    for func in data[key]["board"].keys():
        for var in data[key]["board"][func].keys():
            if type(data[key]["board"][func][var]) == int or type(data[key]["board"][func][var]) == float:
                vars[key][f"{func + '_' + var}"] = data[key]["board"][func][var]
    for var in data[key]["eval_results"].keys():
        vars[key][f"{var}"] = data[key]["eval_results"][var]

In [ ]:
for key in vars[example_key].keys():
    print(key)

In [ ]:
simple_vars = {}

for key in vars.keys():
    simple_vars[key] = {}
    for var in vars[key].keys():
        if "syntax" in var or "board" in var:

            if "nonzero" in var or "dim_count" in var:
                continue

            if "syntax_match" not in var or "pattern_match" not in var:
                continue

            if "syntax_average" in var or "board_average" in var:
                simple_vars[key][var] = vars[key][var]
        elif "find" in var:
            continue
        else:
            simple_vars[key][var] = vars[key][var]

In [ ]:
print(simple_vars)

In [ ]:
max_matching_dict = {}

max_matching_dict["_"] = {"name": "", "count": 0} # any
max_matching_dict["ef16"] = {"name": "", "count": 0}
max_matching_dict["ef8"] = {"name": "", "count": 0}
max_matching_dict["ef4"] = {"name": "", "count": 0}
max_matching_dict["layer=0"] = {"name": "", "count": 0}
max_matching_dict["layer=5"] = {"name": "", "count": 0}

key_of_interest = "board_average_pattern_match_count"
# key_of_interest = "syntax_average_syntax_match_idx_count"

for key in simple_vars.keys():

    for name in max_matching_dict.keys():
        if name in key:
            if simple_vars[key][key_of_interest] > max_matching_dict[name]['count']:
                max_matching_dict[name]['count'] = simple_vars[key][key_of_interest]
                max_matching_dict[name]['name'] = key
    
for name in max_matching_dict.keys():
    print(name, max_matching_dict[name])

In [ ]:
import pandas as pd

# Step 1: Create DataFrame from vars dictionary
df = pd.DataFrame.from_dict(simple_vars, orient='index')

# Step 2: Calculate the correlation matrix
correlation_matrix = df.corr()

# Display the correlation matrix
print(correlation_matrix)

# Optional Step 3: Visualize the correlation matrix using seaborn
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=False, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix of Reported Results')
plt.show()


In [ ]:
x = []
y = []

key1 = "l2_loss"
key2 = "pattern_match_count"

for key in data:
    if "syntax" not in data[key].keys():
        continue
    x.append(data[key]["eval_results"][key1])
    y.append(data[key]["board"]["board_to_piece_state"][key2])

plt.scatter(x, y)
plt.title(f'{key1} vs. {key2}')
plt.xlabel(key1)
plt.ylabel(key2)
plt.grid(True)
plt.show()

In [ ]:
x = []
y = []

key1 = "loss_reconstructed"
key2 = "pattern_match_count"

for key in data:
    if "syntax" not in data[key].keys():
        continue
    x.append(data[key]["eval_results"][key1])
    y.append(data[key]["board"]["board_to_piece_state"][key2])

plt.scatter(x, y)
plt.title(f'{key1} vs. {key2}')
plt.xlabel(key1)
plt.ylabel(key2)
plt.grid(True)
plt.show()

In [ ]:
x = []
y = []

key1 = "syntax_match_idx_count"
key2 = "pattern_match_count"

for key in data:
    if "syntax" not in data[key].keys():
        continue
    x.append(data[key]["syntax"]['find_num_indices'][key1])
    y.append(data[key]["board"]["board_to_piece_state"][key2])

plt.scatter(x, y)
plt.title(f'{key1} vs. {key2}')
plt.xlabel(key1)
plt.ylabel(key2)
plt.grid(True)
plt.show()

In [ ]:
x = []
y = []

key1 = "l0"
key2 = "pattern_match_count"

for key in data:
    if "syntax" not in data[key].keys():
        continue
    x.append(data[key]["eval_results"][key1])
    y.append(data[key]["board"]["board_to_piece_state"][key2])

plt.scatter(x, y)
plt.title(f'{key1} vs. {key2}')
plt.xlabel(key1)
plt.ylabel(key2)
plt.grid(True)
plt.show()

In [ ]:
x = []
y = []

key1 = "l0"
key2 = "syntax_match_idx_count"

for key in data:
    if "syntax" not in data[key].keys():
        continue
    x.append(data[key]["eval_results"][key1])
    y.append(data[key]["syntax"]["find_num_indices"][key2])

plt.scatter(x, y)
plt.title(f'{key1} vs. {key2}')
plt.xlabel(key1)
plt.ylabel(key2)
plt.grid(True)
plt.show()